# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
#np.mean(data[data.race=='w']['call'])


In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b']['call'])


157.0

1. I believe a two sample bootstrapped test is the most appropriate. The test statistic will be the difference in the callback rates of whites and blacks. The central limit theorey applies here, however this is not essential to our analysis because the bootstrapped samples we perform analysis on will conform based on the number of samples.

2. Coventional wisdom is that racial discrimination is a huge issue. This means we would assume the difference in call back rates between white and black sounding names would be different. With this in mind, the null hypothesis will be that racial discrimination in hiring is as or more serious than the data estimates. Meaning, if we reaject the null, we believe the data over estimates the discrimination. 

In [8]:
w = np.array(data[data.race=='w'].call)
b = np.array(data[data.race=='b'].call)

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [50]:
#Define a method to draw bs replicates to analyze 
def draw_bs_reps(data, func, size=1):
    bs_reps= np.empty(size)
    for i in range(size):
        bs_reps[i]= func(np.random.choice(data, size=len(data)))
    return bs_reps
#Use method to create an array of sampled differences to analyze
diff= draw_bs_reps(w,np.mean, size=1000)-draw_bs_reps(b,np.mean, size=1000)
ci= np.percentile(diff, (2.5,97.5))
se=np.sqrt(((np.mean(w)*(1-np.mean(w))/len(w)))+((np.mean(b)*(1-np.mean(b))/len(b))))
error= 1.96*se
ci_f=[diff.mean()-error, diff.mean()+error]
total_mean=np.mean(data.call)
mean_diff=np.mean(w)-np.mean(b)
p= np.sum(diff>=mean_diff)/float(len(diff))


print('The CI according to bootstarapping is:',ci,'According to frequentist methods:', ci_f,'standard error is:', error, 'And the p_value is:', p, 'meaning we will not reject the null')


('The CI according to bootstarapping is:', array([0.01805954, 0.04681725]), 'According to frequentist methods:', [0.01679017893346261, 0.047300991630831256], 'standard error is:', 0.015255406348684322, 'And the p_value is:', 0.51, 'meaning we will not reject the null')


4. The exploratory analysis shows that the null cannot be rejected. This means that discrimination is just as or more pervasive than the data showed. The mean difference recorded by our data was shown be less extreme than nearly half of the sampled means. Additionally, our confidence interval shows that we can expect differences of callbacks between 1-5% if the experiment were replicated.

5. The analysis is not robust enough to make conclusions like that. Other factors such as age, education and working expirience would nead to be included. Also with the introduction of those different variables a different analysis would most likely be necessary. Possibly an OLS regression with Callback as the dependent varible, race as the main coefficient and other factors as controls.